## 07

### Tratamento de dados discrepantes

Remover ou isolar outliers dependendo do copntexto.

### Tratamento de dados faltantes

Identificar e remover linhas/colunas com dados faltantes insignificantes. Imputar valores ausnetes usando estatísticas de tendência central ou modelos preditivos.

## 08

### Codificação de variáveis

Efeturar toda e qualquer transformação necessãria para o funcionamento do modelo, seja ela codificação de variáveis qualitativas (nominais ou ordinais), temporais ou textuais.